In [1]:
import requests as rq
import pandas as pd
from pandas import DataFrame, Series
import re
import csv
import unidecode
import json

In [2]:
home = rq.get("http://www.fifa.com/worldcup/preliminaries/index.html")

In [3]:
from bs4 import BeautifulSoup as bfp
soup = bfp(home.content,'html.parser')

In [55]:
columns = ['ID', 'Name', 'Rank', 'Continent','Flag', 'Site']
counter = 1
ctydatas = DataFrame(columns=columns)

In [5]:
ctys = soup.find(id='qualifiedteamscontainer').find_all('a')

In [6]:
idpattern = re.compile('\d{5}')

In [7]:
for s in ctys:
    site = "http://www.fifa.com/" + s.get('href')
    name = s.img.get("alt")
    flag = "http:" + s.img.get("src")
    _id  = idpattern.search(s.get('href')).group()
    ctydata = DataFrame([[_id, name,0, flag, site]])
    ctydata.columns = columns
    ctydatas = ctydatas.append(ctydata, ignore_index=True)
    counter += 1

In [8]:
ctysoups = {}
for (n, s) in zip(ctydatas['Name'], ctydatas['Site']):
    ctysoups[n] = bfp(rq.get(s).content, "lxml")

In [9]:
ranks = []
for itm in ctysoups.items():
    soup = itm[1]
    i = soup.find(id='profile').find_all("td", {"class":"num"})[2].string
    ranks.append(i)
ctydatas['Rank'] = ranks

In [10]:
ctydatas.to_csv("CountryData.csv", encoding='ascii')


In [61]:
dict_of_dataframe = ctydatas.to_json(orient='table')

In [62]:
dict_of_dataframe = ctydatas.to_json(orient='records')
with open('ctydatas.json', 'w') as fp:
    fp.write(dict_of_dataframe)

In [13]:
ptn1 = re.compile('\.(.*$)')
def cmpname_unicode(sHis, sMine):
    sHis = sHis.lower()
    sMine = sMine.lower()
    sHis = unidecode.unidecode(sHis)
    sMine = unidecode.unidecode(sMine)
    if sHis == sMine:
        return 1
    else:
        cutMine = sMine.split()
        for pts in cutMine:
            if not pts in sHis:
                return -1
        return 1
def cmpname_less(sHis, sMine):
    sHis = sHis.lower().lstrip('-')
    sMine = sMine.lower().lstrip('-')
    if sHis == sMine:
        return 1
    else:
        cutMine = sMine.split()
        for pts in cutMine:
            if not pts in sHis:
                return cmpname_unicode(sHis, sMine)
        return 1
def cmpname_kro(sHis, sMine):
    sHis = sHis.lower()
    sMine = sMine.lower()
    cutHis = sHis.split()
    for pts in cutHis:
        if not pts in sMine:
            return -1
    return 1
def cmpname_unicode1(sHis, sMine):
    sHis = unidecode.unidecode(sHis)
    sMine = unidecode.unidecode(sMine)
    if sHis == sMine:
        return 1
    else:
        cutMine = sMine.split()
        if(cutMine[-1] in sHis):
            return 1
        else:
            return -1
def cmpname_less1(sHis, sMine):
    
    sHis = sHis.lower().replace('-', '')
    sMine = sMine.lower().replace('-', '')
    if sHis == sMine:
        return 1
    else:
        cutMine = sMine.split()
        if(cutMine[-1] in sHis and sHis[0] == sMine[0]):
            return 1
        else:
            return cmpname_unicode1(sHis, sMine)


In [14]:
cmpname_less1("Ahmed El-Shenawy", "AHMED ELSHENAWY" )

1

In [15]:
playersoups = []
for itm in ctysoups.items():
    ctyname = itm[0]
    soup = itm[1]
    lk = soup.find("div", {'class':'team-players-list qualifiers'})
    if lk:
        lk1 = lk.find("div", {'class':'inner'}).get('data-url')
        playersite = "http://www.fifa.com" + lk1
        playersoups.append(bfp(rq.get(playersite).content, "lxml"))
    else:
        print(ctyname)

Russia


In [24]:
cln = ['ID','Fullname','Name','Age','Photo','Nationality','Flag','Overall','Potential','Club','Club Logo','Value','Wage','Special','Acceleration','Aggression','Agility','Balance','Ball control','Composure','Crossing','Curve','Dribbling','Finishing','Free kick accuracy','GK diving','GK handling','GK kicking','GK positioning','GK reflexes','Heading accuracy','Interceptions','Jumping','Long passing','Long shots','Marking','Penalties','Positioning','Reactions','Short passing','Shot power','Sliding tackle','Sprint speed','Stamina','Standing tackle','Strength','Vision','Volleys','CAM','CB','CDM','CF','CM','ID','LAM','LB','LCB','LCM','LDM','LF','LM','LS','LW','LWB','Preferred Positions','RAM','RB','RCB','RCM','RDM','RF','RM','RS','RW','RWB','ST']
offset = 0
miscmp = 0
lastret = []
for idx, playersoup in enumerate(playersoups):
    players = []
    playersName = []
    miscmpNames = []
    lst = list(ctydatas['Name'])
    if lst[idx] == 'Russia':
        offset = 1
    print(lst[idx + offset])
    ctyname = lst[idx + offset]
    #
    #if not ctyname == "Korea Republic":
    #    continue
    #
    for player in playersoup.find_all("div", {'class':'p p-i-no'}):
        p = player.get("data-player-name")
        #print("Looking for", p)
        Found = False
        cmpin = open("full_player_data_final.csv", "rt", encoding='utf-8')
        cmpdata = csv.DictReader(cmpin, fieldnames= cln)
        for cmp in cmpdata:
            if cmp['Nationality'] != ctyname :
                continue
            hp = cmp['Name']
            cpn = cmp['Fullname']
            cmpresult = -1
            if ctyname == "Korea Republic":
                if not cmp['Nationality'] == "Korea Republic":
                    continue
                cmpresult = cmpname_kro(cpn, p)
            else:
                cmpresult = cmpname_less1(cpn, p)
            if cmpresult != -1:
                cmp['Fullname'] = p
                players.append(cmp)
                playersName.append(hp)
                Found = True
                break
        if not Found:
            print("Name [" + p + "] not found correspond")
            if not p in miscmpNames:
                miscmpNames.append(p)
                miscmp += 1
    print(playersName)
    print("Miscmp :", miscmp)
    with open("Players/" + ctyname + 'Players.csv', 'wt', encoding='utf-8') as fout:
        cout = csv.DictWriter(fout, cln,lineterminator='\n')
        cout.writeheader()
        cout.writerows(players)
    for miscmpName in miscmpNames:
        avgdt = pd.read_csv("Players/" + ctyname + 'Players.csv')
        avg = avgdt.mean().drop("ID")
        avg = avg.drop("ID.1")
        avg = avg.astype(int)
        avg = avg.to_dict()
        avg['Fullname'] = miscmpName
        avg['Name'] = miscmpName
        avg['Photo'] = "https://cdn.sofifa.org/player.png"
        avg['Nationality'] = ctyname
        players.append(avg)
    lastret.append(players)
    with open("Players/" + ctyname + 'Players.csv', 'wt', encoding='utf-8') as fout:
        cout = csv.DictWriter(fout, cln,lineterminator='\n')
        cout.writeheader()
        cout.writerows(players)
    csvfile = open("Players/" + ctyname + 'Players.csv', 'r', encoding='utf-8')
    jsonfile = open("Players/" + ctyname + 'Players.json', 'w', encoding='utf-8')
    reader = csv.DictReader( csvfile, cln)
    normal_lst = list(reader)
    length = len(normal_lst)
    csvfile = open("Players/" + ctyname + 'Players.csv', 'r', encoding='utf-8')
    read_again = csv.DictReader( csvfile, cln)
    for idx,row in enumerate(read_again):
        if idx == 0:
            jsonfile.write("[\n")
            continue
        json.dump(row, jsonfile)
        if idx != length - 1:
            jsonfile.write(',\n')
        else:
            jsonfile.write('\n]')
        
    #import shutil
    #import os
    #if os.path.exists("Players/" + ctyname):
    #    shutil.rmtree("Players/" + ctyname)
    #os.mkdir("Players/" + ctyname)
    #for nm in playersName:
    #    shutil.copy("unicode_xlsx_players/img/" + nm + ".png", "Players/" + ctyname + '/' + nm + '.png')

Egypt
Name [HOSSAM ASHOUR] not found correspond
Name [Mahmoud ABDELRAZEK] not found correspond
Name [ABDALLA SAID] not found correspond
Name [WALID SOLIMAN] not found correspond
Name [AYMAN ASHRAF] not found correspond
Name [AHMED TAWFIK] not found correspond
Name [SALEH GOMAA] not found correspond
Name [AHMED MEKKY] not found correspond
Name [MOHAMED ABDEL SHAFI] not found correspond
Name [MOHAMED NAGUIB] not found correspond
Name [TREZEGET] not found correspond
Name [MAHMOUD KAHRABA] not found correspond
Name [AMR GAMAL] not found correspond
Name [SABRI RAHIL] not found correspond
Name [MOAMEN ZAKARIA] not found correspond
Name [MAHDI SOLIMAN] not found correspond
Name [ISLAM GAMAL] not found correspond
Name [HAMADA TOLBA] not found correspond
Name [HOSSAM SALAMA] not found correspond
Name [Mohamed AWAD] not found correspond
Name [Amr MAREI] not found correspond
Name [BASEM MORSI] not found correspond
Name [Héctor CÚPER] not found correspond
['E. El Hadary', 'H. Ghaly', 'S. Ekramy', 

Name [Omid NORAFKAN] not found correspond
Name [Ashkan DEJAGAH] not found correspond
Name [Masoud SHOJAEI] not found correspond
Name [Andranik TIMOTIAN] not found correspond
Name [Pejman MONTAZERI] not found correspond
Name [Ehsan HAJI SAFI] not found correspond
Name [Hashem BEIKZADEH] not found correspond
Name [Khosro HEYDARI] not found correspond
Name [Morteza POURALIGANJI] not found correspond
Name [Omid EBRAHIMI] not found correspond
Name [Reza GHOOCHANNEJAD] not found correspond
Name [Sousha MAKANI] not found correspond
Name [Mehdi SHARIFI] not found correspond
Name [Ali BEIRANVAND] not found correspond
Name [Vorya GHAFOURI] not found correspond
Name [Vahid AMIRI] not found correspond
Name [Mehdi TAREMI] not found correspond
Name [Mohammad KANAANI] not found correspond
Name [Mehdi TORABI] not found correspond
Name [Ezatollah POURGHAZ] not found correspond
Name [Carlos QUEIROZ] not found correspond
['R. Rezaeian', 'K. Ansarifard', 'A. Jahanbakhsh', 'K. Ansarifard', 'K. Ansarifard',

Name [Mesut OEZIL] not found correspond
Name [Andre SCHUERRLE] not found correspond
Name [Marvin PLATTENHARDT] not found correspond
Name [Mario GOETZE] not found correspond
Name [Benedikt HOEWEDES] not found correspond
Name [Thomas MUELLER] not found correspond
Name [Antonio RUEDIGER] not found correspond
Name [Niklas SUELE] not found correspond
['S. Khedira', 'M. GMao mez', 'M. Neuer', 'K. Trapp', 'T. Kroos', 'S. Rudy', 'J. Boateng', 'B. Leno', 'M. ter Stegen', 'S. Mustafi', 'M. Hummels', 'S. Wagner', 'K. Volland', 'M. ter Stegen', 'I. GMei ndoHuo an', 'J. Draxler', 'M. Ginter', 'L. Goretzka', 'M. Meyer', 'J. Brandt', 'J. Kimmich', 'J. Weigl', 'Y. Gerhardt', 'S. Gnabry', 'L. Goretzka', 'A. Younes', 'J. Hector', 'J. Tah', 'K. Bellarabi', 'B. Henrichs', 'K. Demirbay', 'D. Demme', 'L. SanMao ', 'L. Stindl', 'T. Werner', 'A. Schwolow']
Miscmp : 217
Iceland
Name [Holmar EYJOLFSSON] not found correspond
Name [Heimir HALLGRÍMSSON] not found correspond
['O. SkMei lason', 'A. Finnbogason', 'B.

Name [Jose CALDERON] not found correspond
Name [Luis HENRIQUEZ] not found correspond
Name [Blas PEREZ] not found correspond
Name [Jaime PENEDO] not found correspond
Name [Luis TEJADA] not found correspond
Name [Ricardo BUITRAGO] not found correspond
Name [Amilcar HENRIQUEZ] not found correspond
Name [Luis MEJIA] not found correspond
Name [Rolando BLACKBURN] not found correspond
Name [Alberto QUINTERO] not found correspond
Name [Tony TAYLOR] not found correspond
Name [Roberto CHEN] not found correspond
Name [Darwin PINZON] not found correspond
Name [Abdiel ARROYO] not found correspond
Name [Eric DAVIS] not found correspond
Name [Manuel VARGAS] not found correspond
Name [Renan ADDLES] not found correspond
Name [Jean Carlos CEDENO] not found correspond
Name [Alex RODRIGUEZ] not found correspond
Name [Richard DIXON] not found correspond
Name [Ismael DIAZ] not found correspond
Name [Josiel NUNEZ] not found correspond
Name [Roberto NURSE] not found correspond
Name [Edgar BARCENAS] not found 

In [33]:
csvfile = open('Players/RussiaPlayers.csv', 'r', encoding='utf-8')
jsonfile = open('Players/RussiaPlayers.json', 'w', encoding='utf-8')
reader = csv.DictReader( csvfile, cln)
normal_lst = list(reader)
length = len(normal_lst)
csvfile = open('Players/RussiaPlayers.csv', 'r', encoding='utf-8')
read_again = csv.DictReader( csvfile, cln)
for idx,row in enumerate(read_again):
    if idx == 0:
        jsonfile.write("[\n")
        continue
    json.dump(row, jsonfile)
    if idx != length - 1:
        jsonfile.write(',\n')
    else:
        jsonfile.write('\n]')

In [64]:
columns = ['Index','ID', 'Name', 'Rank', 'Continent','Flag', 'Site']
csvfile = open('CountryData.csv', 'r', encoding='utf-8')
jsonfile = open('CountryData.json', 'w', encoding='utf-8')
reader = csv.DictReader( csvfile, columns)
normal_lst = list(reader)
length = len(normal_lst)
csvfile = open('CountryData.csv', 'r', encoding='utf-8')
read_again = csv.DictReader( csvfile, columns)
for idx,row in enumerate(read_again):
    if idx == 0:
        jsonfile.write("[\n")
        continue
    json.dump(row, jsonfile)
    if idx != length - 1:
        jsonfile.write(',\n')
    else:
        jsonfile.write('\n]')
csvfile.close()
jsonfile.close()

In [59]:
csvfile.close()
jsonfile.close()

In [52]:
tdtxt = open('MatchTimePlace.txt', 'r')
tdjson = open('MatchTimePlace.json', 'w')
matchtdlst = []
matchtd = {}
idx = 0
while True:
    row = tdtxt.readline()
    row=row.strip('\n')
    if not row:
        break
    if idx % 6 == 0:
        matchtd = {}
        matchtd['Time'] = row
    elif idx % 6 == 1:
        matchtd['Time'] = matchtd['Time'] + row
    elif idx % 6 == 2:
        matchtd['Team1'] = row
    elif idx % 6 == 3:
        matchtd["ID"] = row
    elif idx % 6 == 4:
        matchtd['Team2'] = row
    else:
        matchtd["Place"] = row
        matchtdlst.append(matchtd)
    idx += 1
print(matchtdlst)
json.dump(matchtdlst, tdjson, sort_keys = True, indent = 4, separators=(',', ': '))

[{'Time': '14 June 2018 18:00 MSK ( UTC+3 ) ', 'Team1': 'Russia  ', 'ID': '1', 'Team2': 'A2', 'Place': 'Luzhniki Stadium , Moscow'}, {'Time': '15 June 2018 17:00 YEKT ( UTC+5 ) ', 'Team1': 'A3', 'ID': '2', 'Team2': 'A4', 'Place': 'Central Stadium , Yekaterinburg'}, {'Time': '19 June 2018 21:00 MSK ( UTC+3 ) ', 'Team1': 'Russia  ', 'ID': '17', 'Team2': 'A3', 'Place': 'Krestovsky Stadium , Saint Petersburg'}, {'Time': '20 June 2018 18:00 MSK ( UTC+3 ) ', 'Team1': 'A4', 'ID': '18', 'Team2': 'A2', 'Place': 'Rostov Arena , Rostov-on-Don'}, {'Time': '25 June 2018 18:00 SAMT ( UTC+4 ) ', 'Team1': 'A4', 'ID': '33', 'Team2': 'Russia', 'Place': 'Cosmos Arena , Samara'}, {'Time': '25 June 2018 17:00 MSK ( UTC+3 ) ', 'Team1': 'A2', 'ID': '34', 'Team2': 'A3', 'Place': 'Volgograd Arena , Volgograd'}, {'Time': '15 June 2018 18:00 MSK ( UTC+3 ) ', 'Team1': 'B3', 'ID': '4', 'Team2': 'B4', 'Place': 'Krestovsky Stadium , Saint Petersburg'}, {'Time': '15 June 2018 21:00 MSK ( UTC+3 ) ', 'Team1': 'B1', 'ID

In [ ]:
dtt = pd.read_csv('Players/AustraliaPlayers.csv')

In [50]:
tdjson.close()

In [ ]:
dtt = dtt.drop(["Unnamed: 0"],axis=1)

In [ ]:
s = dtt.mean()

In [ ]:
s = s.astype(int)

In [ ]:
dtbu['Fullname'] = "Alex GERSBACH"

In [ ]:
from collections import OrderedDict, defaultdict
s.to_dict()